In [1]:
from crud_script import CRUDScript


In [2]:
# 1: Imports and CRUD module
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# import MongoDB CRUD wrapper
from crud_script import CRUDScript

In [3]:
# 2: Load data from MongoDB via your CRUD module

# -- AAC credentials
USER            = 'root'
PASSWORD        = 'ItA0pKPZnu'
HOST            = 'nv-desktop-services.apporto.com'
PORT            = 31016
DB_NAME         = 'aacdb'
COLLECTION_NAME = 'outcomes'


crud = CRUDScript(USER, PASSWORD, HOST, PORT, DB_NAME, COLLECTION_NAME)
raw_data = crud.read({})      # retrieve all documents
df = pd.DataFrame(raw_data)
# drop Mongo’s internal _id
if '_id' in df.columns:
    df = df.drop(columns=['_id'])

# rename fields
df = df.rename(columns={
    'rescue_type':     'RescueType',    # if you created this; otherwise map outcome_type
    'breed':           'Breed',
    'location_lat':    'lat',
    'location_long':   'lon'
})


# Drop Mongo’s internal _id and rename columns for clarity
if '_id' in df.columns:
    df = df.drop(columns=['_id'])
df = df.rename(columns={
    'rescue_type': 'RescueType',
    'breed':       'Breed',
    'latitude':    'lat',      
    'longitude':   'lon'
})

print(df.columns.tolist())

['', 'age_upon_outcome', 'animal_id', 'animal_type', 'Breed', 'color', 'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'lat', 'lon', 'age_upon_outcome_in_weeks']


In [4]:
# 3: Initialize Dash app

app = dash.Dash(
    __name__,
    external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
server = app.server

In [5]:
# 4: Layout 

app.layout = html.Div([
    html.Div([
        html.Img(src='/',
                 style={'height': '80px'}),
        html.H2('Grazioso Salvare Search-&-Rescue Dashboard',
                style={'display': 'inline-block', 'marginLeft': '20px'}),
    ], style={'display': 'flex', 'alignItems': 'center'}),

    html.Div('Developer: Corderro Artz',
             style={'fontStyle': 'italic', 'marginBottom': '20px'}),

    # Radio buttons for rescue-type filter
    dcc.RadioItems(
        id='rescue-filter',
        options=[
            {'label': 'All',                             'value': 'All'},
            {'label': 'Water Rescue',                    'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue',   'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
        ],
        value='All',
        labelStyle={'display': 'inline-block', 'marginRight': '15px'}
    ),

    # Interactive data table
    dash_table.DataTable(
        id='results-table',
        columns=[{'name': c, 'id': c} for c in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        style_table={'overflowX': 'auto'},
    ),

    # Charts
    dcc.Graph(id='map-graph'),
    dcc.Graph(id='breed-count-graph'),
],
style={'padding': '20px'})


In [6]:
@app.callback(
    Output('results-table', 'data'),
    Output('map-graph', 'figure'),
    Output('breed-count-graph', 'figure'),
    Input('rescue-filter', 'value')
)
def update_dashboard(selected_type):
    # filter DataFrame
    if selected_type == 'All':
        dff = df
    else:
        dff = df[df['RescueType'] == selected_type]

    # 1) update table
    table_data = dff.to_dict('records')

    # 2) geolocation map
    fig_map = px.scatter_geo(
        dff,
        lat='lat',
        lon='lon',
        hover_name='Breed',
        title='Candidate Dog Locations',
        scope='usa'
    )

    # 3) breed distribution bar chart
    counts = dff['Breed'].value_counts().reset_index()
    counts.columns = ['Breed', 'Count']
    fig_breed = px.bar(
        counts,
        x='Breed',
        y='Count',
        title='Breed Distribution'
    )

    return table_data, fig_map, fig_breed


In [ ]:
# 6: Run the app

if __name__ == '__main__':
    app.run_server(port=8085, debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8085/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
